In [21]:
%matplotlib inline

##Import Data

In [2]:
import epi_prediction
src_dir = '/Users/johnbonihalab/data/chuyu/modalities'
control_filter = lambda file_name: 'CON' in file_name
patient_filter = lambda file_name: 'PAT' in file_name

In [3]:
epi_paths = epi_prediction.get_epi_paths(src_dir, patient_filter, control_filter)


In [4]:
mod_paths = epi_paths['dmean']

In [5]:
mod_paths

cons    [/Users/johnbonihalab/data/chuyu/modalities/dm...
pats    [/Users/johnbonihalab/data/chuyu/modalities/dm...
Name: dmean, dtype: object

In [6]:
labels = len(mod_paths['pats']) * [1] + len(mod_paths['cons']) * [0]

In [7]:
len(labels)

68

##Load White Matter Mask

In [8]:
white_matter = epi_prediction.SimpleMasker('masks/white.nii', threshold=.2)

In [9]:
mat = white_matter.transform_many(mod_paths['pats'] + mod_paths['cons'], verbose=True)

--------------------------------------------------------------------
####################################################################

##Retrieve the Voxels

__Retrieving Important Voxels Requires Linear Classifier__

In [10]:
from nilearn.input_data import NiftiMasker
import numpy as np
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [11]:
n_folds = 6
svc = SVC()
anova_svc_lin = Pipeline( [('anova', SelectKBest(k=500)), ('svc', svc)] )
anova_svc_lin.set_params(svc__kernel='linear')
cv = StratifiedKFold(labels, n_folds=6)
labels_arr = np.array(labels)

In [12]:
C_range_lin = np.logspace(-3, 3, 7)
#k_range = np.linspace(500,2000,3).astype(int)
k_range=[500]

total_runs = len(C_range_lin) * 2 * len(k_range) * 2

scorer = epi_prediction.verbose_scorer(total_runs)
grid_lin = GridSearchCV(anova_svc_lin, param_grid={"svc__C": C_range_lin,"anova__k" :k_range}, cv=cv, scoring = scorer)

grid_lin.fit(mat, labels_arr)

----------------------------
##########################################

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[1 1 ..., 0 0], n_folds=6, shuffle=False, random_state=None),
       error_score='raise',
       estimator=Pipeline(steps=[('anova', SelectKBest(k=500, score_func=<function f_classif at 0x10b0a65f0>)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'anova__k': [500], 'svc__C': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring=make_scorer(verbose_score_fn), verbose=0)

In [13]:
grid_lin.grid_scores_

[mean: 0.58889, std: 0.19303, params: {'anova__k': 500, 'svc__C': 0.001},
 mean: 0.69020, std: 0.13056, params: {'anova__k': 500, 'svc__C': 0.01},
 mean: 0.69265, std: 0.14039, params: {'anova__k': 500, 'svc__C': 0.10000000000000001},
 mean: 0.64751, std: 0.12043, params: {'anova__k': 500, 'svc__C': 1.0},
 mean: 0.64751, std: 0.12043, params: {'anova__k': 500, 'svc__C': 10.0},
 mean: 0.64751, std: 0.12043, params: {'anova__k': 500, 'svc__C': 100.0},
 mean: 0.64751, std: 0.12043, params: {'anova__k': 500, 'svc__C': 1000.0}]

In [14]:
anova_svc_lin.set_params(svc__C=.1, anova__k=500) #best k is using 2000, but pre C of C was 500

Pipeline(steps=[('anova', SelectKBest(k=500, score_func=<function f_classif at 0x10b0a65f0>)), ('svc', SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [15]:
svc = anova_svc_lin.named_steps['svc']
anova = anova_svc_lin.named_steps['anova']

In [16]:
def fit(train):
    anova_svc_lin.fit(mat[train], labels_arr[train])

def coeffs_fn():
    return np.abs(white_matter.inv_transform(anova.inverse_transform(svc.coef_)))

coeffs = epi_prediction.calc_coeffs(cv, fit, coeffs_fn)

In [17]:
coeffs.shape

(91, 109, 91)

In [19]:
reload(epi_prediction)

import IPython
from IPython.html.widgets import interact, interactive
from IPython.html.widgets.widget_float import FloatSlider

title='dMean'
def plot_bw(base_b):
    epi_prediction.plot_bw_coeffs(coeffs, white_matter._mask_image.get_affine(), title,
                                 base_brightness=base_b)

base_b_slider = FloatSlider(min=0, max=1, step=0.1, value=.7)
w=interactive(plot_bw,base_b=base_b_slider)
IPython.display.display(w)

In [20]:
epi_prediction.plot_coeffs(coeffs, white_matter._mask_image.get_affine(), title)
epi_prediction.plot_bw_coeffs(coeffs, white_matter._mask_image.get_affine(), title, 
                           output_file=title+'_bw', base_brightness=.7)

##Generate Linear Confusion Matrix

In [22]:
cv_info_lin, cv_info_train_lin = epi_prediction.verbose_cv(mat, labels_arr, anova_svc_lin, n_folds=6, verbose=False)

####
####
####
####
####
####


In [23]:
cv_info_lin.confusion_matrix()

array([[27,  9],
       [10, 22]])

In [24]:
cv_info_lin.normalized_confusion_matrix()

array([[ 0.75  ,  0.25  ],
       [ 0.3125,  0.6875]])

In [25]:
epi_prediction.plot_confusion_matrix(cv_info_lin.normalized_confusion_matrix(), "dMean Confusion Matrix")

In [26]:
cv_info_lin.avg_f1_score()

0.69343434343434351

In [27]:
cv_info_lin.avg_sensitivity_score()

0.69444444444444453

In [28]:
cv_info_lin.avg_specificity_score()

0.75

### Normalize Matrix Z-score

In [26]:
from scipy.stats.mstats import zscore

X = mat.copy()
for col in xrange(X.shape[1]):
    if col % 50000 == 0:
        print '{} columns fin.'.format(col)
    X[:,col] = zscore(X[:,col]) 
    X[:,col] = X[:,col] / np.sqrt( sum( [ x**2 for x in X[:,col] ] ) )

0 columns fin.
50000 columns fin.
100000 columns fin.


In [27]:
mat.shape

(68, 134263)

<b><h3> Grid Search params := ( $k_{best}$,C ,$T_{mask}$, norm= (True || False) ) </h3></b>

In [28]:
print mat.shape

(68, 134263)


In [29]:
mat_normalized = X

In [30]:
grid_lin_normalized = GridSearchCV(anova_svc_lin, param_grid={"svc__C": C_range_lin}, cv=cv, scoring = scorer)
grid_lin_normalized.fit(mat_normalized, labels_arr)

####

//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
//anaconda/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:111: UserWarning: Features [84317] are constant.
  UserWarning)


######################################

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[1 1 ..., 0 0], n_folds=6, shuffle=False, random_state=None),
       error_score='raise',
       estimator=Pipeline(steps=[('anova', SelectKBest(k=2000, score_func=<function f_classif at 0x10c603578>)), ('svc', SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'svc__C': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring=make_scorer(verbose_score_fn), verbose=0)

In [31]:
grid_lin_normalized.grid_scores_

[mean: 0.25840, std: 0.11905, params: {'svc__C': 0.001},
 mean: 0.68284, std: 0.18130, params: {'svc__C': 0.01},
 mean: 0.67213, std: 0.22921, params: {'svc__C': 0.10000000000000001},
 mean: 0.75330, std: 0.17163, params: {'svc__C': 1.0},
 mean: 0.75330, std: 0.17163, params: {'svc__C': 10.0},
 mean: 0.75330, std: 0.17163, params: {'svc__C': 100.0},
 mean: 0.75330, std: 0.17163, params: {'svc__C': 1000.0}]

### White Matter Threshold

In [32]:
all_fs = mod_paths['pats'] + mod_paths['cons']

In [33]:
import sys

import nilearn as nil

wm_image = nib.load('masks/white.nii')

def verbose_load(f):
    sys.stdout.write('#')
    return nil.image.resample_img(f, 
                                  target_shape=wm_image.shape, 
                                  target_affine=wm_image.get_affine())

print('-' * len(all_fs))
    
all_data = np.vstack([verbose_load(f).get_data().flatten() for f in all_fs])

--------------------------------------------------------------------
####################################################################

In [34]:
all_data.shape

(68, 902629)

In [35]:
reload(epi_prediction)
white_matter_pipe = epi_prediction.SimpleMaskerPipeline(threshold=0)

In [36]:
normalizer = epi_prediction.NormalizerPipeline(normalize_flag=False)

In [37]:
wm_norm_anova_svc = Pipeline( [('masker', white_matter_pipe),
                              ('anova', SelectKBest(k=500)), 
                              ('svc', svc)] )

In [38]:
param_grid={"svc__C": [.01, .1, 1, 10, 100],
            "masker__threshold" :[.2, .5, .7, .9],
            "anova__k": [50, 250, 500]
           }

print("-" * 2 * 5  * 2 * 4 * 2 * 3)
grid_lin = GridSearchCV(wm_norm_anova_svc, param_grid=param_grid, cv=cv, scoring = scorer)
grid_lin.fit(all_data, labels_arr)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
########################################################################################################################################################################################################################################################################################################################################################################

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[1 1 ..., 0 0], n_folds=6, shuffle=False, random_state=None),
       error_score='raise',
       estimator=Pipeline(steps=[('masker', SimpleMaskerPipeline(threshold=0)), ('anova', SelectKBest(k=500, score_func=<function f_classif at 0x10c603578>)), ('svc', SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'masker__threshold': [0.2, 0.5, 0.7, 0.9], 'anova__k': [50, 250, 500], 'svc__C': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring=make_scorer(verbose_score_fn), verbose=0)

In [39]:
grid_lin.grid_scores_

[mean: 0.53903, std: 0.19375, params: {'masker__threshold': 0.2, 'anova__k': 50, 'svc__C': 0.01},
 mean: 0.64510, std: 0.15356, params: {'masker__threshold': 0.2, 'anova__k': 50, 'svc__C': 0.1},
 mean: 0.61976, std: 0.20442, params: {'masker__threshold': 0.2, 'anova__k': 50, 'svc__C': 1},
 mean: 0.72006, std: 0.11017, params: {'masker__threshold': 0.2, 'anova__k': 50, 'svc__C': 10},
 mean: 0.71471, std: 0.12134, params: {'masker__threshold': 0.2, 'anova__k': 50, 'svc__C': 100},
 mean: 0.05392, std: 0.12423, params: {'masker__threshold': 0.5, 'anova__k': 50, 'svc__C': 0.01},
 mean: 0.66204, std: 0.12022, params: {'masker__threshold': 0.5, 'anova__k': 50, 'svc__C': 0.1},
 mean: 0.72871, std: 0.11339, params: {'masker__threshold': 0.5, 'anova__k': 50, 'svc__C': 1},
 mean: 0.74040, std: 0.12838, params: {'masker__threshold': 0.5, 'anova__k': 50, 'svc__C': 10},
 mean: 0.65637, std: 0.18860, params: {'masker__threshold': 0.5, 'anova__k': 50, 'svc__C': 100},
 mean: 0.00000, std: 0.00000, para

In [40]:
grid_lin.best_score_

0.74040346907993948

In [41]:
grid_lin.best_params_

{'anova__k': 50, 'masker__threshold': 0.5, 'svc__C': 10}